# Tests de Chi-cuadrado

En la lección anterior presentamos el marco de las pruebas estadísticas de hipótesis y la prueba t para investigar las diferencias entre variables numéricas. Ahora nos centraremos en una prueba estadística habitual para variables categóricas: el test de chi-cuadrado.

# Test de bondad de ajuste Chi-cuadrado

En nuestro estudio de las pruebas t, introdujimos la prueba t unidireccional para comprobar si la media de una muestra difiere de la media esperada (población). el test de bondad de ajuste chi-cuadrado es un análogo de la prueba t unidireccional para variables categóricas: comprueba si la distribución de los datos categóricos de la muestra coincide con una distribución esperada. Por ejemplo, podría utilizar un test de bondad de ajuste chi-cuadrado para comprobar si la demografía racial de los miembros de su iglesia o colegio coincide con la de toda la población de EE.UU. o si las preferencias del navegador en el ordenador de sus amigos coinciden con las de los usuarios de Internet en su conjunto.

Cuando se trabaja con datos categóricos, los valores en sí no sirven de mucho para las pruebas estadísticas porque categorías como "hombre", "mujer" y "otros" no tienen significado matemático. Las pruebas que tratan con variables categóricas se basan en recuentos de variables en lugar de en el valor real de las propias variables.

Generemos algunos datos demográficos falsos para EE.UU. y Minnesota y realicemos el test de bondad de ajuste chi-cuadrado para comprobar si son diferentes:

In [1]:
import numpy as np
import pandas as pd
import scipy.stats as stats

In [2]:
national = pd.DataFrame(["white"]*100000 + ["hispanic"]*60000 +\
                        ["black"]*50000 + ["asian"]*15000 + ["other"]*35000)
           

minnesota = pd.DataFrame(["white"]*600 + ["hispanic"]*300 + \
                         ["black"]*250 +["asian"]*75 + ["other"]*150)

national_table = pd.crosstab(index=national[0], columns="count")
minnesota_table = pd.crosstab(index=minnesota[0], columns="count")

print( "Nacional")
print(national_table)
print(" ")
print( "Minnesota")
print(minnesota_table)

National
col_0      count
0               
asian      15000
black      50000
hispanic   60000
other      35000
white     100000
 
Minnesota
col_0     count
0              
asian        75
black       250
hispanic    300
other       150
white       600


Los test de chi-cuadrado se basan en el denominado estadístico chi-cuadrado. El estadístico chi-cuadrado se calcula con la siguiente fórmula:

$$ sum(\frac{(observed-expected)^2}{expected}) $$

En la fórmula, observado es el recuento real observado para cada categoría y esperado es el recuento esperado basado en la distribución de la población para la categoría correspondiente. Para ilustrarlo, calculemos el estadístico chi-cuadrado para nuestros datos:

In [3]:
observed = minnesota_table

national_ratios = national_table/len(national)  # Obtener ratios de población

expected = national_ratios * len(minnesota)   # Obtener recuentos previstos

chi_squared_stat = (((observed-expected)**2)/expected).sum()

print(chi_squared_stat)

col_0
count    18.194805
dtype: float64


*Nota: El test de chi-cuadrado supone que ninguno de los recuentos esperados es inferior a 5.

De forma similar a la prueba t, en la que comparamos el estadístico de la prueba t con un valor crítico basado en la distribución t para determinar si el resultado es significativo, en el test de chi-cuadrado comparamos el estadístico del test de chi-cuadrado con un valor crítico basado en la [distribución chi-cuadrado](https://es.wikipedia.org/wiki/Distribuci%C3%B3n_%CF%87%C2%B2). La abreviatura de la biblioteca scipy para la distribución chi-cuadrado es chi2. Utilicemos este conocimiento para encontrar el valor crítico para un nivel de confianza del 95% y comprobar el valor p de nuestro resultado:

In [4]:
crit = stats.chi2.ppf(q = 0.95, # Hallar el valor crítico para una confianza del 95%*.
                      df = 4)   # Df = número de categorías de variables - 1

print("Valor crítico")
print(crit)

p_value = 1 - stats.chi2.cdf(x=chi_squared_stat,  # Hallar el valor p
                             df=4)
print("P value")
print(p_value)

Critical value
9.487729036781154
P value
[0.00113047]


Dado que nuestro estadístico chi-cuadrado supera el valor crítico, rechazaríamos la hipótesis nula de que las dos distribuciones son iguales.

Puede realizar un test de bondad de ajuste chi-cuadrado automáticamente utilizando la función de scipy **scipy.stats.chisquare()**:

In [5]:
stats.chisquare(f_obs= observed,   # Conjunto de recuentos observados
                f_exp= expected)   # Matriz de recuentos previstos

Power_divergenceResult(statistic=array([18.19480519]), pvalue=array([0.00113047]))

Los resultados de las pruebas coinciden con los valores calculados anteriormente.

# Test de independencia Chi-cuadrado

[Independencia](https://es.wikipedia.org/wiki/Independencia_(probabilidad)) es un concepto clave en probabilidad que describe una situación en la que conocer el valor de una variable no dice nada sobre el valor de otra. Por ejemplo, el mes en que naciste probablemente no te dice nada sobre el navegador que utilizas, por lo que esperaríamos que el mes de nacimiento y la preferencia de navegador fueran independientes. Por otro lado, tu mes de nacimiento puede estar relacionado con tu rendimiento deportivo en la escuela, por lo que el mes de nacimiento y el rendimiento deportivo pueden no ser independientes.

El test de independencia chi-cuadrado comprueba si dos variables categóricas son independientes. La prueba de independencia se utiliza habitualmente para determinar si variables como la educación, las opiniones políticas y otras preferencias varían en función de factores demográficos como el sexo, la raza y la religión. Vamos a generar algunos datos de sondeo de votantes falsos y a realizar una prueba de independencia:

In [6]:
np.random.seed(10)

# Muestrear datos aleatoriamente con probabilidades fijas
voter_race = np.random.choice(a= ["asian","black","hispanic","other","white"],
                              p = [0.05, 0.15 ,0.25, 0.05, 0.5],
                              size=1000)

# Muestrear datos aleatoriamente con probabilidades fijas
voter_party = np.random.choice(a= ["democrat","independent","republican"],
                              p = [0.4, 0.2, 0.4],
                              size=1000)

voters = pd.DataFrame({"race":voter_race, 
                       "party":voter_party})

voter_tab = pd.crosstab(voters.race, voters.party, margins = True)

voter_tab.columns = ["democrat","independent","republican","row_totals"]

voter_tab.index = ["asian","black","hispanic","other","white","col_totals"]

observed = voter_tab.iloc[0:5,0:3]   # Obtener tabla sin totales para uso posterior
voter_tab

,democrat,independent,republican,row_totals
asian,21,7,32,60
black,65,25,64,154
hispanic,107,50,94,251
other,15,8,15,38
white,189,96,212,497
col_totals,397,186,417,1000


Ten en cuenta que no hemos utilizado los datos sobre la raza para generar los datos sobre los partidos, por lo que las variables son independientes.

Para un test de independencia, utilizamos la misma fórmula chi-cuadrado que utilizamos para el test de bondad de ajuste. La principal diferencia es que tenemos que calcular los recuentos esperados de cada celda en una tabla bidimensional en lugar de una tabla unidimensional. Para obtener el recuento esperado de una celda, multiplicamos el total de filas de esa celda por el total de columnas de esa celda y se divide por el número total de observaciones. Podemos obtener rápidamente los recuentos esperados para todas las celdas de la tabla tomando los totales de fila y los totales de columna de la tabla, realizando un producto exterior sobre ellos con la función **np.outer()** y dividiendo por el número de observaciones:

In [7]:
expected =  np.outer(voter_tab["row_totals"][0:5],
                     voter_tab.loc["col_totals"][0:3]) / 1000

expected = pd.DataFrame(expected)

expected.columns = ["democrat","independent","republican"]
expected.index = ["asian","black","hispanic","other","white"]

expected

,democrat,independent,republican
asian,23.820,11.160,25.020
black,61.138,28.644,64.218
hispanic,99.647,46.686,104.667
other,15.086,7.068,15.846
white,197.309,92.442,207.249


Ahora podemos seguir los mismos pasos que antes para calcular el estadístico chi-cuadrado, el valor crítico y el valor p:

In [8]:
chi_squared_stat = (((observed-expected)**2)/expected).sum().sum()

print(chi_squared_stat)

7.169321280162059


*Nota: Llamamos a .sum() dos veces: una vez para obtener las sumas de las columnas y una segunda vez para sumar las sumas de las columnas, devolviendo la suma de toda la tabla 2D.*

In [9]:
crit = stats.chi2.ppf(q = 0.95, # Hallar el valor crítico para una confianza del 95%*.
                      df = 8)   # *

print("Critical value")
print(crit)

p_value = 1 - stats.chi2.cdf(x=chi_squared_stat,  # Hallar el valor p
                             df=8)
print("P value")
print(p_value)

Critical value
15.50731305586545
P value
0.518479392948842


*Nota: Los grados de libertad para un test de independencia son iguales al producto del número de categorías en cada variable menos 1. En este caso tenemos una tabla de 5x3 por lo que df = 4x2 = 8.*

Al igual que con el test de bondad de ajuste, podemos utilizar scipy para realizar una prueba de independencia rápidamente. Utilice la función **stats.chi2_contingency()** para realizar una prueba de independencia automáticamente dada una tabla de frecuencias de los recuentos observados:

In [10]:
stats.chi2_contingency(observed= observed)

(7.169321280162059, 0.518479392948842, 8, array([[ 23.82 ,  11.16 ,  25.02 ],
        [ 61.138,  28.644,  64.218],
        [ 99.647,  46.686, 104.667],
        [ 15.086,   7.068,  15.846],
        [197.309,  92.442, 207.249]]))

La salida muestra el estadístico chi-cuadrado, el valor p y los grados de libertad seguidos de los recuentos esperados.

Como era de esperar, dado el elevado valor p, el resultado de la prueba no detecta una relación significativa entre las variables.

# Wrap Up

Los test de Chi-cuadrado permiten investigar las diferencias en las distribuciones de variables categóricas con las mismas categorías y la dependencia entre variables categóricas.

# Siguiente lección: [ANOVA](6-ANOVA.ipynb)